# 🧪 Test DermLIP Model Loading from HuggingFace

This notebook tests loading the DermLIP model from HuggingFace Hub.

**Model**: `redlessone/DermLIP_PanDerm-base-w-PubMed-256`

**Note**: Native OpenCLIP loading fails due to `pretrain_path` in the HF config. We use **manual weight loading** instead.


## 1️⃣ Install Dependencies


In [ ]:
# Install required packages
%pip install -q open_clip_torch safetensors huggingface_hub

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 2️⃣ Load DermLIP Visual Encoder (Manual Method)

This downloads weights from HuggingFace and loads them into a ViT-B-16 model.


In [ ]:
import open_clip
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

print(f"OpenCLIP version: {open_clip.__version__}")
print("="*60)
print("Loading DermLIP Visual Encoder (Manual Method)")
print("="*60)

# Step 1: Download weights from HuggingFace
print("\n1. Downloading weights from HuggingFace...")
weights_path = hf_hub_download(
    repo_id="redlessone/DermLIP_PanDerm-base-w-PubMed-256",
    filename="open_clip_model.safetensors"
)
print(f"   ✅ Downloaded to: {weights_path}")

# Step 2: Create base ViT-B-16 model (same architecture as DermLIP)
print("\n2. Creating base ViT-B-16 model...")
model, _, preprocess = open_clip.create_model_and_transforms(
    'ViT-B-16',
    pretrained=None  # No pretrained weights
)
print(f"   ✅ Model created")

# Step 3: Load DermLIP weights
print("\n3. Loading DermLIP weights...")
state_dict = load_file(weights_path)
print(f"   Total keys in weights: {len(state_dict)}")

# Extract visual encoder weights only
visual_state = {
    k.replace("visual.", ""): v 
    for k, v in state_dict.items() 
    if k.startswith("visual.")
}
print(f"   Visual encoder keys: {len(visual_state)}")

# Load into model
missing, unexpected = model.visual.load_state_dict(visual_state, strict=False)
print(f"   Missing keys: {len(missing)}")
print(f"   Unexpected keys: {len(unexpected)}")

# Step 4: Test forward pass
print("\n4. Testing forward pass...")
model.eval()
dummy_img = torch.randn(1, 3, 224, 224)
with torch.no_grad():
    features = model.encode_image(dummy_img)
print(f"   ✅ Image features shape: {features.shape}")

print("\n" + "="*60)
print("✅ SUCCESS: DermLIP visual encoder loaded correctly!")
print("="*60)


## 3️⃣ Verify Model Architecture


In [ ]:
# Verify architecture matches DermLIP config
print("Model Architecture Verification")
print("="*60)

# Expected from DermLIP open_clip_config.json:
# image_size: 224, layers: 12, width: 768, patch_size: 16
expected = {
    "image_size": 224,
    "layers": 12, 
    "width": 768,
    "patch_size": 16,
    "output_dim": 512
}

# Check visual encoder
visual = model.visual
print(f"\n✅ Visual encoder output dim: {visual.output_dim}")
print(f"   (Expected: {expected['output_dim']})")

# Count transformer layers
if hasattr(visual, 'transformer') and hasattr(visual.transformer, 'resblocks'):
    num_layers = len(visual.transformer.resblocks)
    print(f"✅ Transformer layers: {num_layers}")
    print(f"   (Expected: {expected['layers']})")

# Check embedding dimension
if hasattr(visual, 'conv1'):
    embed_dim = visual.conv1.out_channels
    print(f"✅ Embedding dimension: {embed_dim}")
    print(f"   (Expected: {expected['width']})")

print("\n" + "="*60)
print("Architecture matches DermLIP visual encoder! ✅")
print("="*60)


## 4️⃣ Test on GPU (if available)


In [ ]:
if torch.cuda.is_available():
    print("Testing on GPU...")
    device = torch.device('cuda')
    model = model.to(device)
    
    # Test batch inference
    batch_size = 4
    dummy_batch = torch.randn(batch_size, 3, 224, 224).to(device)
    
    with torch.no_grad(), torch.amp.autocast('cuda'):
        features = model.encode_image(dummy_batch)
    
    print(f"✅ Batch inference on GPU successful!")
    print(f"   Input shape: {dummy_batch.shape}")
    print(f"   Output shape: {features.shape}")
    print(f"   GPU memory used: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
else:
    print("⚠️ No GPU available, skipping GPU test")


## ✅ Summary

If all tests pass, the DermLIP visual encoder is loaded correctly and ready for PanDerm training!

**Next step**: Upload updated `src/models_panderm.py` to Google Drive and run `Train_PanDerm_A100.ipynb`
